In [100]:
import numpy as np
import cv2
import os
from sklearn.cluster import KMeans

In [45]:
def read_dataset_metadata(dir_path):
    dataset = {}
    for filename in os.listdir(dir_path):
        if os.path.isdir(dir_path + filename):
            dataset[filename] = []
            for img in os.listdir(os.path.join(dir_path + filename)):
                dataset[filename].append(os.path.join(dir_path,filename,img))
    return dataset

def get_frequent_object(dataset,count=3):
    """
    dataset : Dictionary object returned by the function 'read_dataset'
    
    Returns a list of objects with their frequency in descending order
    """
    key_count = {}
    for key in dataset.keys():
        key_count[key] = len(dataset[key])
    
    
    object_keys = []
    for key, value in sorted(key_count.iteritems(), key=lambda (k,v): (v,k), reverse=True):
        object_keys.append(key)
        count-=1
        if count == 0:
            break
    return object_keys
    

In [46]:
dataset_metadata = read_dataset_metadata('./101_ObjectCategories/')

In [47]:
frequent_objects = get_frequent_object(dataset_metadata,5)
print frequent_objects

['airplanes', 'Motorbikes', 'BACKGROUND_Google', 'Faces_easy', 'Faces']


In [40]:
#The output of the above cell shows the 5 most frequent objects
#In the interest of choosing three different categories as diverse from each other as possible
#We choosse 'airplanes', 'Motorbikes' and 'Faces_easy' classes

In [49]:
categories = ['airplanes', 'Motorbikes','Faces_easy']

new_dataset_metadata = {}
for category in categories:
    new_dataset_metadata[category] = dataset_metadata[category]

In [69]:
def make_dataset(metadata):
    try:
        dataset = []
        for key in metadata.keys():
            for img_path in metadata[key]:
                img = cv2.imread(img_path,0)
                dataset.append((img,key))
        dataset = np.array(dataset)
        np.save('dataset',dataset)
        return 'dataset.npy'
    except:
        raise "Error in creating Dataset File"

In [70]:
dataset_path = make_dataset(new_dataset_metadata)

In [71]:
dataset = np.load(dataset_path)

In [80]:
test_image = dataset[0][0]
sift = cv2.xfeatures2d.SIFT_create()


In [101]:
key_points = sift.detect(test_image,None)

In [102]:
key_points_cart = [key_point.pt for key_point in key_points]

In [122]:
kmeans = KMeans(n_clusters=5).fit(key_points_cart)

In [123]:
kmeans.cluster_centers_

array([[  85.856209  ,   89.91530022],
       [ 332.68444824,   76.70238133],
       [ 190.2274754 ,   81.86299515],
       [  69.57211974,   34.23409211],
       [ 262.50506427,   65.79946755]])

In [137]:
def make_bag_of_words(dataset,code_size = 8):
    try:
        bow_data = []
        for img_data in dataset:
            img = img_data[0]
            label = img_data[1]

            key_points = sift.detect(img,None)
            key_points_cart = [key_point.pt for key_point in key_points]
            kmeans = KMeans(n_clusters=code_size).fit(key_points_cart)

            bow_data.append((kmeans.cluster_centers_,label))

        bow_data = np.array(bow_data)
        np.save('bag_of_words',bow_data)
        return 'bag_of_words.npy'
    except:
        raise "Error in creating Bag of Words File"

In [138]:
bow_data_path = make_bag_of_words(dataset,5)

In [139]:
bow_data = np.load(bow_data_path)

In [143]:
bow_data[0]

array([ array([[ 264.22552665,   64.26056104],
       [  85.856209  ,   89.91530022],
       [ 332.68444824,   76.70238133],
       [  69.57211974,   34.23409211],
       [ 191.50533896,   82.19230837]]),
       'airplanes'], dtype=object)